# TPSmay21 Categorical Features 
While exploring the data. I noticed that the max value of the features have small numbers. So maybe it can improve the prediction if converts features from numerical to categorical. I did it with CatBoost and indeed it improved the score.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import catboost
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss

df = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')

In [ ]:
new_df = df.copy()

## CatBoost

In [ ]:
clf = CatBoostClassifier(random_state = 0)

In [ ]:
def create_model(the_categorical_features_names, the_df, bool_full, drop_columns):
    X = the_df.drop(drop_columns, axis=1)
    X = the_df.drop("target", axis=1)
    y = the_df.target
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0) 
    
    for i in the_categorical_features_names:
        X[i] = X[i].astype('category')
    if (bool_full):
        clf.fit(X,y, cat_features=the_categorical_features_names, eval_set=(X_test, y_test),logging_level = 'Silent')
    else:
        clf.fit(X_train,y_train, cat_features=the_categorical_features_names, eval_set=(X_test, y_test), logging_level = 'Silent')
        
        predictions = clf.predict_proba(X_test)
        predictions
        log_loss_result = log_loss(y_test, predictions)
        round(log_loss_result, 5)

In [ ]:
categorical_features_names = list(new_df.columns)
categorical_features_names.remove("target")
# categorical_features_names.remove("id")

Calculate max values of the features

In [ ]:
features_max_df = pd.DataFrame(columns=["Name", "MaxValue"])
f_features_names = list(df.columns)
f_features_names.remove('id')
f_features_names.remove('target')

In [ ]:
for i, d in enumerate(f_features_names):
    feature_max_value = df[d].max()
    features_max_df.loc[i] = [d, feature_max_value]

features_max_df.head(2)        

In [ ]:
def create_features_below_list(below_number):
    return list(features_max_df[features_max_df.MaxValue < below_number]["Name"])

In [ ]:
features_below_10 = create_features_below_list(10)
features_below_20 = create_features_below_list(20)
features_below_30 = create_features_below_list(30)
features_below_40 = create_features_below_list(40)
print("features_below_10 = ", features_below_10)

In [ ]:
# Convert all features into 'category'
# create_model(categorical_features_names, new_df, True, ["id"]) # Public Score: 1.09196
# create_model(categorical_features_names, new_df, True, []) # Public Score:1.08975

# Convert only features with max value below ten
# create_model(features_below_10, new_df, True, []) # Public Score: 1.09522
# create_model(features_below_20, new_df, True, []) # Public Score: 1.09321
# create_model(features_below_30, new_df, True, []) # Public Score: 1.09379
create_model(features_below_40, new_df, True, [])

## Submission

In [ ]:
test_df = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
submission_df = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')

In [ ]:
X = test_df.copy()
test_predictions = clf.predict_proba(X)
test_predictions


In [ ]:
df_test_predictions = pd.DataFrame(test_predictions, columns=["Class_1", "Class_2", "Class_3", "Class_4"])
df_test_predictions.head(2)

In [ ]:
submission_df["Class_1"] = df_test_predictions["Class_1"]
submission_df["Class_2"] = df_test_predictions["Class_2"]
submission_df["Class_3"] = df_test_predictions["Class_3"]
submission_df["Class_4"] = df_test_predictions["Class_4"]
submission_df.head(2)

In [ ]:
submission_df.to_csv('submission4.csv', index=False)